In [ ]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

In [ ]:


import gensim
from gensim import utils
import numpy as np
from nltk import word_tokenize
from nltk import download

from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


In [ ]:
word_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)


In [ ]:
download('punkt')
def preprocess(text5):
    text5 = text5.lower()
    doc = word_tokenize(text5)
    return doc



In [ ]:
def document_vector(word2vec_model, doc):
    doc = [word for word in doc if word in word2vec_model.vocab]
    if(len(doc)==0):
      return [0]*300
    return np.mean(word2vec_model[doc], axis=0)



In [ ]:
file1 = open('Subj.x.train.txt', 'r',encoding='ISO-8859-1') 
texts = file1.readlines() 

In [ ]:
file3 = open('Subj.x.test.txt', 'r',encoding='ISO-8859-1') 
texts2 = file3.readlines()

In [ ]:
from sklearn.preprocessing import LabelEncoder

file1 = open('Subj.y.train.txt', 'r') 
Lines = file1.readlines() 
count = 0
target1=[]
for line in Lines: 
    target1.append(line)
le = LabelEncoder()
le.fit(["SUBJ\n","OBJ\n"])
list(le.classes_)
labels=le.transform(target1)
labels[:29]

In [ ]:
texts, y = texts, labels
corpus = [preprocess(text) for text in texts]

In [ ]:
test_corpus = [preprocess(text3) for text3 in texts2]

In [ ]:

x =[]
for doc in corpus: 
      x.append(document_vector(word_model, doc))
X = np.array(x)
X.shape, len(y)


In [ ]:
X[90]

In [ ]:
test_x =[]
for doc1 in test_corpus: 
    test_x.append(document_vector(word_model, doc1))
test_X = np.array(test_x) 
np.savetxt('subjtest_documents_vectors2.txt', test_X)
test_X.shape


In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
train_x=X
train_y=y
X_train, X_val, Y_train, Y_val = train_test_split(train_x, train_y, test_size=0.20, random_state=42)
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
Y_train=tf.convert_to_tensor(Y_train, dtype=tf.float32)
X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
Y_val=tf.convert_to_tensor(Y_val, dtype=tf.float32)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv=6, max_iter=1000,random_state=0,multi_class='ovr').fit(X_train, Y_train)
score = clf.score(X_val, Y_val)

print("Accuracy:", score)

In [ ]:
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
scale.fit(X_train)
X_train=scale.transform(X_train)
X_val=scale.transform(X_val)

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=1000,random_state=0,multi_class='ovr')
classifier.fit(X_train, Y_train)
score = classifier.score(X_val, Y_val)

print("Accuracy:", score)

In [ ]:
X_test=tf.convert_to_tensor(test_X, dtype=tf.float32)

In [ ]:
ypred=clf.predict(X_test)

In [ ]:
COUNT=0
for x in ypred:
  if(x==0):
    COUNT+=1

print(COUNT)

In [ ]:
print(ypred[:300])

In [ ]:
print(ypred[:50])

In [ ]:
model = Sequential()
model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
history=model.fit(X_train, Y_train, validation_data=(X_val,Y_val), epochs=5, batch_size=10)

In [ ]:
o=model.evaluate(X_val,Y_val)

In [ ]:
y_pred=model.predict_classes(X_test)

In [ ]:
y_pred2=model.predict_classes(X_val)

In [ ]:
print(y_pred)

In [ ]:
count=0
for x in ypred:
  if(x==0):
    count+=1
print(count)

In [ ]:
Y_test=[]
for i in range(len(ypred)):
  if(ypred[i]==0):
    Y_test.append('OBJ')
  else:
    Y_test.append('SUBJ')


In [ ]:
print(Y_test)

In [ ]:
f1 = open("Subj.y.test.txt", "w")
for y in Y_test:
  y1=str(y)
  f1.write(str(y1))
  f1.write("\n")

In [ ]:
f2=open("Subj.y.test.txt","r")
count=1
for x in f2:
  print(count,x)
  count+=1